In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing  as Smooth
from statsmodels.graphics.tsaplots import plot_pacf as PACF
from statsmodels.graphics.tsaplots import plot_acf as ACF
from statsmodels.tsa.stattools import adfuller as ADF
from statsmodels.tools.eval_measures import aic as AIC, bic as BIC
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA 
from statsmodels.tsa.arima_model import ARIMA 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from sklearn.metrics import mean_squared_error as MSE
import numpy as np
qqplot=sm.qqplot
ljungbox=sm.stats.acorr_ljungbox

import pandas as pd
from time import time
from IPython.display import clear_output
from urllib.request import urlopen
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')


In [2]:
blockchain=pd.read_csv('./blockchain.csv')
print(f"BITCION BLOCKCHAIN CONTAINING {len(blockchain)} BLOCKS")

tx=pd.read_csv('./tx.csv')
print(f"BITCION TRANSACTION HISTORY HAVING {len(tx)} TRANSACTIONS")

#inp_out=pd.read_csv('./inp_out.csv')   
#print(f"BITCION INCOMING AND OUTOING TRANSACTION HISTORY HAVING {len(inp_out)} SAMPLES")

BITCION BLOCKCHAIN CONTAINING 2208 BLOCKS
BITCION TRANSACTION HISTORY HAVING 2600215 TRANSACTIONS


In [3]:
def delete_block(indicies,blockchain,tx,inp_out):
 B=blockchain[blockchain['block_index'].isin(indicies)]
 blockchain=blockchain.drop(B.index)
 indicies=B.block_index
 B=tx[tx['block_index'].isin(indicies)]
 tx=tx.drop(B.index)
 indicies=B['index']
 B=inp_out[inp_out['transaction-id'].isin(indicies)]
 inp_out=inp_out.drop(B.index)
 return (blockchain,tx,inp_out)

def save(blockchain):
 blockchain.to_csv('../blockchain.csv',index=False)


In [4]:
## Taking sample in the block
#indicies=list(blockchain.iloc[50:]['block_index'])
#blockchain,tx,inp_out=delete_block(indicies,blockchain,tx,inp_out)

In [5]:
inp_out=inp_out.drop(inp_out[inp_out['value']!=inp_out['value']].index)
inp_out=inp_out.drop(inp_out[inp_out['value']==0].index)
inp_out=inp_out.astype({'value':'int64'})

In [6]:
DF=inp_out.groupby('transaction-id').sum('value')
X=pd.merge(DF,tx,how='left',left_on='transaction-id', right_on='index')
P=X.groupby('block_index').sum('value')

In [7]:
indicies=list(P[P['value']!=-6.25e+08].index)
blockchain,tx,inp_out=delete_block(indicies,blockchain,tx,inp_out)

blockchain['time']=pd.to_datetime(blockchain['time'],unit='s')
indicies=blockchain[blockchain.time.dt.month>3]['block_index']
blockchain,tx,inp_out=delete_block(indicies,blockchain,tx,inp_out)


In [8]:
tmp=inp_out.groupby('transaction-id').sum('value')
tmp=tmp[tmp['value']<0]
tmp=pd.merge(tx,tmp,how='right',left_on='index',right_on='transaction-id')
tmp.reset_index()
tmp=tmp[['block_index','value']]
tmp.columns=['block_index','fee']

blockchain=pd.merge(tmp,blockchain,on='block_index')

In [9]:
tmp=tx.groupby('block_index').count()
tmp=tmp.drop('index',axis=1)
tmp=tmp.reset_index()
tmp=tmp.iloc[:,:2]
tmp.columns=['block_index','ntx']

blockchain=pd.merge(tmp,blockchain,on='block_index')

In [10]:
tmp=inp_out.groupby(['transaction-id','addr']).sum('value').reset_index()
tmp['value']=np.abs(tmp['value'])
tmp=tmp.groupby('transaction-id').sum('value')
tmp=pd.merge(tx,tmp,how='right',left_on='index',right_on='transaction-id')
tmp=tmp.groupby('block_index').sum('value').reset_index()
tmp=tmp[['block_index','value']]
tmp.columns=['block_index','Transmission']

blockchain=pd.merge(tmp,blockchain,on='block_index')

In [11]:
tmp=inp_out
tmp=pd.merge(tx,tmp,how='right',left_on='index',right_on='transaction-id')[['addr','block_index']]

tmp['num_transmission']=1
tmp=tmp.groupby(['block_index','addr']).sum('num_transmission').reset_index()

tmp['num_addr']=1
tmp=tmp.groupby('block_index').sum()

blockchain=pd.merge(tmp.reset_index(),blockchain,on='block_index')

In [12]:
blockchain['HashRate']=blockchain['time'].diff() / np.timedelta64(1, 's')
blockchain=blockchain.iloc[1:]
blockchain=blockchain.astype({'HashRate':'int64'})

In [13]:
save(blockchain)

In [14]:
tmp=inp_out.groupby(['transaction-id','addr']).sum('value').reset_index()
tmp=tmp.groupby('transaction-id').sum('value')
tmp=pd.merge(tx,tmp,how='right',left_on='index',right_on='transaction-id')
tmp=tmp.groupby('block_index').sum('value').reset_index()
tmp=tmp[['block_index','value']]
tmp.columns=['block_index','Transmission']